幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

if not os.path.exists(StorePath+'/'+TITLE+'feature/'): 
    os.makedirs(StorePath+'/'+TITLE+'feature/')
FeaturePath=StorePath+'/'+TITLE+'feature/'

センサデータの読み込み

In [3]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [4]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

# temp file 
import tempfile

%matplotlib inline

In [5]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [6]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [7]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

In [8]:
windowNum = 256
slidingNum = windowNum/8

wind=GetWindowFrame('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making


In [9]:
wind.shape

(33676, 256)

# Fourier Transformation

## Energy

In [10]:
def FastVector_Energy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
    # calcurate eq
    f = lambda x: np.real(x)**2+np.imag(x)**2
        
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    SqrF=np.array([
            np.real(fftwind[i,l])**2+np.imag(fftwind[i,l])**2
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] )
        ])
    SqrF = SqrF.reshape(fftwind.shape)    
    # calcurate Squared Energy
    SqrF=SqrF/SqrF[0,:].size
    Output = SqrF**0.5
    return Output

# Frequency

In [11]:
def FastVector_Frequency(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    print wind.shape
    
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    print fftwind.shape

    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    print fftwind.shape
    freq = np.array([np.arctan2(np.imag(fftwind[i,l]),np.real(fftwind[i,l]))
                     for i in range( fftwind.shape[0] ) 
                     for l in range( fftwind.shape[1] ) ])
    freq = freq.reshape(fftwind.shape)    
    return freq

In [12]:
FastVector_Frequency('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making
(33676, 256)
(33676, 256)
(33676, 126)


array([[-2.23333261, -0.49089132,  1.73135164, ..., -0.30988429,
         0.64142871, -0.60591236],
       [-1.66039142,  1.1243195 , -1.96200826, ...,  1.25773398,
        -0.5394879 ,  1.91979035],
       [-0.98857872,  3.02318647,  0.1850124 , ..., -1.49213272,
        -2.89431385,  0.56944015],
       ..., 
       [ 3.08361934,  0.42039599,  1.87214907, ..., -1.90954865,
        -2.47987151,  0.57469526],
       [-0.52463096,  1.63281432, -1.15083388, ...,  1.69695187,
         2.23532294, -1.19602007],
       [ 0.43528963,  3.13674861,  1.53463999, ..., -1.44356796,
        -0.8016174 , -1.89793012]])

# Entropy

In [13]:
def FastVector_Entropy(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    ###########################
    ###    calcurate FFT    ###
    ###########################
    fftwind = np.fft.fft(wind)  # calcurate fast fourier Transfromation
    fftwind = fftwind[:,1:fftwind.shape[1]/2-1] # Orthogonal and Colossus removed
    
    # calcurate eq
    f = lambda x: np.real(x)**2+np.imag(x)**2
        
    # windowFrame(raw wave) 2d -> windowFrame(power) 1d
    SqrF=np.array([
            np.real(fftwind[i,l])**2+np.imag(fftwind[i,l])**2
            for i in range( fftwind.shape[0] )
            for l in range( fftwind.shape[1] )
        ])
    SqrF = SqrF.reshape(fftwind.shape)
    
    # calcurate Squared Energy
    SqrF=SqrF/SqrF[0,:].size
    print SqrF.shape
    
    ##############################
    ###    calcurate Entropy   ###
    ##############################
    P = np.array([SqrF[i]/SqrF[i,:].sum() for i in range(SqrF.shape[0])])
    print P.shape
    P = P.reshape(SqrF.shape)
    
    P_logP = np.array([ P[i,l]*np.log(P[i,l]) \
                      for i in range(P.shape[0]) for l in range(P.shape[1])])
    print P_logP.shape
    P_logP = P_logP.reshape(P.shape)
    
    Entropy = np.array([
            -1*np.sum( P_logP[i,:] ) for i in range(P.shape[0])])
    
    return Entropy

In [14]:
e=FastVector_Entropy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making
(33676, 126)
(33676, 126)
(4243176,)


In [15]:
# this length is num of windowFrame
e.shape[0]

33676

# Kullback leibler Divergence

In [16]:
def FastVector_KLD(dataName, dataX, dataY, wWidth, sWidth, PATH):
    """
        Calculates Kullback–Leibler divergence
        input dataX dataY -> 1d Vector
    """
    func_kld =lambda p,q: np.sum(p * np.log(p / q))
    
    KLD = np.array([ func_kld(dataX[i,:],dataY[i,:])\
                    for i in range(dataX.shape[0])])
    return KLD

In [17]:
# for FastVector_KLD, make distribution
distX=FastVector_Energy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
distY=FastVector_Energy('AccY', Sensor1['AccY'], windowNum, slidingNum, WindowDataPath)

this data had finished making
this data had finished making


In [18]:
distX.shape

(33676, 126)

In [19]:
a=FastVector_KLD('Sensor1',distX,distY, windowNum, slidingNum, WindowDataPath)
a.shape

(33676,)

# Jensen Shannon Divergence

In [20]:
def FastVector_JSD(dataName, dataX, dataNameY, dataY, wWidth, sWidth, PATH):
    
    func_kld =lambda p,q: np.sum(p * np.log(p / q))
    func_jsd =lambda p,q: 0.5 * func_kld(p, (p+q)/2) + 0.5 * func_kld(q, (p+q)/2)

    JSD = np.array([ func_jsd(dataX[i,:],dataY[i,:])\
                    for i in range(dataX.shape[0])])
    return JSD

In [22]:
distX=FastVector_Energy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
distY=FastVector_Energy('AccY', Sensor1['AccY'], windowNum, slidingNum, WindowDataPath)

a=FastVector_JSD('Sensor1',distX,'Sensor1',distY, windowNum, slidingNum, WindowDataPath)
a.shape

this data had finished making
this data had finished making


(33676,)

In [23]:
windowNum = 256
slidingNum = windowNum/32

SaveFileName=Sensor1['Name']+'_AccX'+'_Win='+str(windowNum).zfill(4)+'_Sld='+str(slidingNum).zfill(4)

import time

np.savez(FeaturePath+SaveFileName+'_Energy',data=\
     FastVector_Energy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
time.sleep(0.1)

np.savez(FeaturePath+SaveFileName+'_Entropy',data=\
     FastVector_Entropy('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
time.sleep(0.1)

np.savez(FeaturePath+SaveFileName+'_Frequency',data=\
     FastVector_Frequency('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)
)
time.sleep(0.1)


this data had finished making
this data had finished making
(134702, 126)
(134702, 126)
(16972452,)
this data had finished making
(134702, 256)
(134702, 256)
(134702, 126)
